<a href="https://colab.research.google.com/github/Enjeru-105/Google-Colab/blob/main/Proyecto_Final_Recordatorio_y_Env%C3%ADo_de_Felicitaciones_de_Cumplea%C3%B1os.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q streamlit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 649.3 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 54.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 71.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 5.1 MB/s eta 0:00:00


In [10]:
%%writefile app.py

import streamlit as st
from datetime import datetime, date
import smtplib, ssl, csv, os, random

# Datos de los contactos que se guarden
class Contacto:
  def __init__(self, nombre, fecha_nacimiento, email):
    self.nombre = nombre
    self.fecha_nacimiento = fecha_nacimiento
    self.email = email

  # Funciones
  def calcular_proximo_cumple(self):
    hoy = date.today()
    cumple_este_ano = date(hoy.year, self.fecha_nacimiento.month, self.fecha_nacimiento.day)
    # Verifica si ya cumplió años este año, antes de hoy
    if cumple_este_ano < hoy:
      cumple_este_ano = date(hoy.year + 1, self.fecha_nacimiento.month, self.fecha_nacimiento.day)

    # Regresa la cantidad de días que faltan para su cumpleaños
    dias_faltantes = (cumple_este_ano - hoy).days
    return dias_faltantes, cumple_este_ano

#Gestor de archivo de contactos
class GestorContactos:
  def __init__(self, archivo_contactos='contactos.csv'):
    self.archivo_contactos = archivo_contactos
    self.contactos = []
    self.cargar_contactos()

  def cargar_contactos(self):
    if os.path.exists(self.archivo_contactos):
      with open(self.archivo_contactos, mode='r', newline='') as file:
        reader = csv.DictReader(file)
        for row in reader:
          fecha_nac = datetime.strptime(row['fecha_nacimiento'], '%Y-%m-%d').date()
          self.contactos.append(Contacto(row['nombre'], fecha_nac, row['email']))

  def guardar_contactos(self):
    with open(self.archivo_contactos, mode='w', newline='') as file:
      writer = csv.DictWriter(file, fieldnames=['nombre', 'fecha_nacimiento', 'email'])
      writer.writeheader()
      for contacto in self.contactos:
        writer.writerow({'nombre': contacto.nombre, 'fecha_nacimiento': contacto.fecha_nacimiento.strftime('%Y-%m-%d'), 'email': contacto.email})

  def agregar_contacto(self, nombre, fecha_nacimiento, email):
    nuevo_contacto = Contacto(nombre, fecha_nacimiento, email)
    self.contactos.append(nuevo_contacto)
    self.guardar_contactos()
    return nuevo_contacto

  def eliminar_contacto(self, nombre):
    self.contactos = [c for c in self.contactos if c.nombre != nombre]
    self.guardar_contactos()

#Gestor de archivo de mensajes
class GestorMensajes:
  def __init__(self, archivo_mensajes='mensajes.csv'):
    self.archivo_mensajes = archivo_mensajes
    self.mensajes = []
    self.cargar_mensajes()

  def cargar_mensajes(self):
    if os.path.exists(self.archivo_mensajes):
      with open(self.archivo_mensajes, mode='r', newline='') as file:
        reader = csv.DictReader(file)
        self.mensajes = [row['mensaje'] for row in reader]

  def guardar_mensajes(self):
    with open(self.archivo_mensajes, mode='w', newline='') as file:
      writer = csv.DictWriter(file, fieldnames=['mensaje'])
      writer.writeheader()
      for mensaje in self.mensajes:
        writer.writerow({'mensaje': mensaje})

  def agregar_mensaje(self, mensaje):
    self.mensajes.append(mensaje)
    self.guardar_mensajes()

  def eliminar_mensaje(self, mensaje):
    self.mensajes.remove(mensaje)
    self.guardar_mensajes()

#Gestor de cumpleaños
class Gestor_cumpleanos:
  def __init__(self, gestor_contactos):
    self.contactos = gestor_contactos.contactos

  def obtener_proximos_cumpleanos(self, limite=5):
    contactos_con_dias = []
    for contacto in self.contactos:
      dias, fecha = contacto.calcular_proximo_cumple()
      contactos_con_dias.append((dias, fecha, contacto))

    contactos_con_dias.sort(key=lambda x: x[0])
    return contactos_con_dias[:limite]

  def verificar_cumpleanos_hoy(self):
    hoy = date.today()
    cumpleaneros = []
    for contacto in self.contactos:
      if hoy.month == contacto.fecha_nacimiento.month and hoy.day == contacto.fecha_nacimiento.day:
        cumpleaneros.append(contacto)
    return cumpleaneros

#Gestor de envíos de mensajes
class EnviarMensaje:
  @staticmethod
  def enviar_felicitacion(contacto, mensaje, config_email):
    #Intenta mandar el mensaje
    try:
      context = ssl.create_default_context()###
      with smtplib.SMTP(config_email['smtp_server'], config_email['port']) as server:
        server.ehlo()
        server.starttls(context=context)
        server.ehlo()
        server.login(config_email['sender_email'], config_email['password'])
        #mensaje
        subject = f"Feliz Cumpleaños {contacto.nombre}"
        body = f"{mensaje}\n\nSaludos,\n{config_email['sender_name']}"
        message = f"Subject: {subject}\n\n{body}"
        server.sendmail(config_email['sender_email'], contacto.email, message.encode('utf-8'))
      return True, "Felicitación enviada con éxito"
    except Exception as e:
      return False, f"Error al enviar la felicitación: {str(e)}"

# Main Interfaz con Streamlit
st.title('🎂 Gestor de Cumpleaños')
if 'gestor_contactos' not in st.session_state:
  st.session_state['gestor_contactos'] = GestorContactos()
if 'gestor_mensajes' not in st.session_state:
  st.session_state['gestor_mensajes'] = GestorMensajes()

gestor_cumpleanos = Gestor_cumpleanos(st.session_state.gestor_contactos)
#Menú lateral
opciones = ["🏠 Inicio", "👥 Gestionar Contactos", "💌 Gestionar Mensajes", "⚙️ Configurar Email", "📨 Enviar Felicitaciones"]
opcion = st.sidebar.radio("Menú", opciones)

if opcion == "🏠 Inicio":
  st.subheader("Proximos Cumpleaños")
  proximos = gestor_cumpleanos.obtener_proximos_cumpleanos(5)
  if not proximos:
    st.write("No hay contactos registrados o no hay cumpleaños próximos.")
  else:
    for dias, fecha, contacto in proximos:
      st.write(f"**{contacto.nombre}** - {fecha.strftime('%d/%m/%Y')} - Faltan {dias} días")

  cumpleaneros_hoy = gestor_cumpleanos.verificar_cumpleanos_hoy()
  if cumpleaneros_hoy:
    st.subheader("🎉 ¡Cumpleaños Hoy!")
    for contacto in cumpleaneros_hoy:
      st.success(f"¡Hoy es el cumpleaños de {contacto.nombre}! 🎈")

elif opcion == "👥 Gestionar Contactos":
  st.subheader("Gestionar Contactos")
  with st.expander("➕ Agregar Nuevo Contacto"):
    with st.form("form_contacto"):
      #Ingresar datos de nuevo contacto
      nombre = st.text_input("Nombre")
      fecha_nac = st.date_input("Fecha de Nacimiento", min_value=date(1900, 1, 1))
      email = st.text_input("Email")

      if st.form_submit_button("Guardar Contacto"):
        if nombre and email:
          st.session_state.gestor_contactos.agregar_contacto(nombre, fecha_nac, email)
          st.success("Contacto agregado correctamente!")
          st.rerun()
        else:
          st.error("Por favor, complete todos los campos.")
  #Contactos guardados
  st.subheader("Lista de Contactos")
  if not st.session_state.gestor_contactos.contactos:
    st.write("No hay contactos registrados.")
  else:
    for contacto in st.session_state.gestor_contactos.contactos:
      col1, col2 = st.columns([4, 1])
      with col1:
        st.write(f"**{contacto.nombre}** - {contacto.fecha_nacimiento.strftime('%d/%m')} - {contacto.email}")
      with col2:
        if st.button("🗑️", key=f"del_{contacto.nombre}"):
          st.session_state.gestor_contactos.eliminar_contacto(contacto.nombre)
          st.success("Contacto eliminado correctamente!")
          st.rerun()

elif opcion == "💌 Gestionar Mensajes":
  st.subheader("Mensajes de Felicitación")
  with st.expander("➕ Agregar Nuevo Mensaje"):
    with st.form("form_mensaje"):
      nuevo_mensaje = st.text_area("Nuevo Mensaje")
      if st.form_submit_button("Guardar Mensaje") and nuevo_mensaje:
        st.session_state.gestor_mensajes.agregar_mensaje(nuevo_mensaje)
        st.success("Mensaje agregado correctamente!")
        st.rerun()
  #Mensajes Guardados
  st.subheader("Mensajes Guardados")
  if not st.session_state.gestor_mensajes.mensajes:
    st.write("No hay mensajes registrados.")
  else:
    for i, mensaje in enumerate(st.session_state.gestor_mensajes.mensajes):
      col1, col2 = st.columns([4, 1])
      with col1:
        st.write(mensaje)
      with col2:
        if st.button("🗑️", key=f"del_msg_{i}"):
          st.session_state.gestor_mensajes.eliminar_mensaje(mensaje)
          st.success("Mensaje eliminado correctamente!")
          st.rerun()

elif opcion == "⚙️ Configurar Email":
  st.subheader("Configuración de Email")
  if 'email_config' not in st.session_state:
    st.session_state.email_config = {'sender_name': '', 'sender_email': '', 'password': '', 'smtp_server': 'smtp.gmail.com', 'port': 587}
  #Agregar datos para envío
  with st.form("form_email_config"):
    sender_name = st.text_input("Tu Nombre", value=st.session_state.email_config['sender_name'])
    sender_email = st.text_input("Tu Email", value=st.session_state.email_config['sender_email'])
    password = st.text_input("Contraseña", type="password", value=st.session_state.email_config['password'])
    smtp_server = st.text_input("Servidor SMTP", value=st.session_state.email_config['smtp_server'])
    port = st.number_input("Puerto SMTP", min_value=1, max_value=65535, value=st.session_state.email_config['port'])

    if st.form_submit_button("Guardar Configuración"):####
      st.session_state.email_config = {'sender_name': sender_name, 'sender_email': sender_email, 'password': password, 'smtp_server': smtp_server, 'port': port}
      st.success("Configuración guardada correctamente!")

elif opcion == "📨 Enviar Felicitaciones":
  st.subheader("Enviar Felicitaciones")
  if 'email_config' not in st.session_state or not st.session_state.email_config['sender_email']:
    st.warning("Primero configura tus datos de email en la sección de configuración.")
  elif not st.session_state.gestor_contactos.contactos:
    st.warning("No hay contactos registrados.")
  elif not st.session_state.gestor_mensajes.mensajes:
    st.warning("No hay mensajes registrados.")
  else:
    cumpleaneros_hoy = gestor_cumpleanos.verificar_cumpleanos_hoy()
    if cumpleaneros_hoy:
      st.write("### Felicitaciones Automáticas para Hoy")
      for contacto in cumpleaneros_hoy:
        with st.expander(f"Enviar a {contacto.nombre}"):
          if st.button(f"Enviar Felicitación a {contacto.nombre}", key=f"auto_{contacto.nombre}"):
            mensaje = random.choice(st.session_state.gestor_mensajes.mensajes)
            success, message = EnviarMensaje.enviar_felicitacion(contacto, mensaje, st.session_state.email_config)
            if success:
              st.success(message)
            else:
              st.error(message)

    st.write("### Enviar Felicitaciones Manualmente")
    contacto_seleccionado = st.selectbox("Selecciona un contacto", [c.nombre for c in st.session_state.gestor_contactos.contactos])
    mensaje_seleccionado = st.selectbox("Selecciona un mensaje", st.session_state.gestor_mensajes.mensajes)

    if st.button("Enviar Felicitación"):
      contacto = next(c for c in st.session_state.gestor_contactos.contactos if c.nombre == contacto_seleccionado)
      success, message = EnviarMensaje.enviar_felicitacion(contacto, mensaje_seleccionado, st.session_state.email_config)
      if success:
        st.success(message)
      else:
        st.error(message)

Overwriting app.py


In [3]:
!npm install localtunnel

⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦
added 22 packages in 3s
⠦
⠦3 packages are looking for funding
⠦  run `npm fund` for details
⠦

In [14]:
!streamlit run app.py &>/content/logs.txt & npx localtunnel --port 8501 & curl ipv4.icanhazip.com

34.73.220.235
⠙your url is: https://poor-cars-rescue.loca.lt
